In [12]:
import cv2
import numpy as np
from PIL import Image 
import pickle
import os
print("MisterMaker")

MisterMaker


In [2]:
# We Have to Run this Cell for Once Only as it will generate the Pickle File for once and all.
namelist = []
file_name = "namelist.pkl"
f = open(file_name,'wb')

pickle.dump(namelist,f)
f.close()



# For Dataset generation

In [15]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

with open('namelist.pkl', 'rb') as f:
    namelist= pickle.load(f)
    
name = input("Enter Full Name for the Data Generation: ")
namelist.append(name)
id = namelist.index(name)


def face_cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    for(x,y,w,h) in faces:
        face_cropped = img[y:y+h,x:x+w]
        return face_cropped
    
cap = cv2.VideoCapture(0)
# cap.set(3, 640) # set video width
# cap.set(4, 480) # set video height
img_id = 0

while True:
    ret,my_frame = cap.read()
    
    if face_cropped(my_frame) is not None:
        img_id+=1
        
        face = cv2.resize(face_cropped(my_frame),(450,450))
        
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
  
        file_path="data/"+name+"."+str(id)+"."+str(img_id)+".jpg"
        

        cv2.imwrite(file_path,face)
        cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),2)
        cv2.imshow("Generated Face",face)
        
    if cv2.waitKey(1)==13 or int(img_id) == 100:
        break
with open('namelist.pkl','wb') as f:
    pickle.dump(namelist,f)
    
    

cap.release()
cv2.destroyAllWindows()

Enter Full Name for the Data Generation: female


# Training the Model 

In [8]:
def trainer():
    # Path for face image database
    path = 'data'

    recognizer = cv2.face.LBPHFaceRecognizer_create() 
    detector = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_default.xml")

    # function to get the images and label data
    def imagenames(path):

        imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
        face_samples=[]
        faceids = []

        for imagePath in imagePaths:

            pilimage = Image.open(imagePath).convert('L') 
            img_numpy = np.array(pilimage,'uint8')

            id = int(os.path.split(imagePath)[-1].split(".")[1])
            faces = detector.detectMultiScale(img_numpy)

            for (x,y,w,h) in faces:
                face_samples.append(img_numpy[y:y+h,x:x+w])
                faceids.append(id)

        return face_samples,faceids

    print ("\nTraining Started.\n")
    faces,faceids = imagenames(path)
    recognizer.train(faces, np.array(faceids))

   
    recognizer.write('trainer/trainer.yml') 

   
    print("{0} faces trained. Exiting Training Program".format(len(np.unique(faceids))))

In [16]:
trainer()


Training Started.

3 faces trained. Exiting Training Program


# Face Recognition using LPBH Algorithm 


In [17]:
def faceRecognition():
    print('\nStarting Recognizer....')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer/trainer.yml')
 
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_default.xml")

    font = cv2.FONT_HERSHEY_SIMPLEX

    # Starting realtime video capture
    cam = cv2.VideoCapture(0)
    
    # Define min window size to be recognized as a face
    minW = 0.1*cam.get(3)
    minH = 0.1*cam.get(4)

    with open('namelist.pkl', 'rb') as f:
        names = pickle.load(f)

    while True:

        ret, img = cam.read()
        

        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale( 
            gray,
            scaleFactor = 1.2,
            minNeighbors = 5,
            minSize = (int(minW), int(minH)),
           )

        for(x,y,w,h) in faces:

            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

            # Check if confidence is less them 100 ==> "0" is perfect match 
            if (confidence < 100):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))

            cv2.putText(img, str(id), (x+5,y-5), font, 1, (0,0,0), 2)
            cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (0,0,0), 1)  

        cv2.imshow('camera',img) 

        k = cv2.waitKey(13) & 0xff 
        if k == 27:
            break

  
    print("\nExiting Recognizer.")
    cam.release()
    cv2.destroyAllWindows()

In [18]:
faceRecognition()


Starting Recognizer....

Exiting Recognizer.
